# Исследование постов на Hacker News
---
В данном проекте мы хотим сравнить два типа постов с сайта [Hacker News](https://news.ycombinator.com/): посты с вопросами (начинаются с **Ask HN**) и посты с демонтсрацией чего-либо(начинаются с **Show HN**). Узнаем, какие из них набираются больше комментариев, а также зависит ли популярность постов от времени, когда они были созданы.

Для проекта используем данный набор данных [отсюда](https://www.kaggle.com/hacker-news/hacker-news-posts).

Первым делом преобразуем эти данные во вложеный список.

In [1]:
def save_as_list(dataset):
    opened_dataset = open(dataset)
    from csv import reader 
    read_dataset = reader(opened_dataset)
    dataset_as_list = list(read_dataset)
    return dataset_as_list

hn = save_as_list('hacker_news.csv')
for row in hn[0:5]:
    print(row)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']
['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


Первая строка из набора данных содержит заголовки. Отделим её и назначим отдельной переменной, удалив при этом из основной. 

In [2]:
header = hn[0]
print(header)
hn = hn[1:]
for row in hn[0:5]:
    print(row)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']
['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']
['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']


Разделим все посты из данного набора на три группы:
* Посты, которые начинаются с `ask hn`
* Посты, которые начинаются с `show hn`
* Все остальные посты

Также посмотим на их количество.

In [3]:
ask_list = []
show_list = []
other_list = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_list.append(row)
    elif title.lower().startswith('show hn'):
        show_list.append(row)
    else:
        other_list.append(row)
        
print('Кол-во постов ask hn: ', len(ask_list),
     '\nКол-во постов show hn: ', len(show_list),
     '\nКол-во остальных постов: ', len(other_list))

Кол-во постов ask hn:  1744 
Кол-во постов show hn:  1162 
Кол-во остальных постов:  17194


Посмотрим среднее количество комментариев для постов с `ask hn` и `show hn`

In [4]:
def avg_comments(dataset):
    comments_qty = 0
    for row in dataset:
        comments_qty += int(row[4])
    avg_com = comments_qty / len(dataset)    
    return avg_com

avg_ask_comments = avg_comments(ask_list)
avg_show_comments = avg_comments(show_list)

print('Среднее кол-во комментов с ask hn', avg_ask_comments,
     '\nСреднее кол-во комментов с show hn', avg_show_comments)

Среднее кол-во комментов с ask hn 14.038417431192661 
Среднее кол-во комментов с show hn 10.31669535283993


Среднее количество комментариев в постах с вопросами больше: **14** против **10**.

Возможно это связано с тем, что пользователи усваивают информацию из постов `show hn` в "ридонли". Вопросы же подразумевают ответ, который пользователи стараются дать.

И раз уж вопросы более популярны, то мы сфокусируемся только на `ask_list`. Посмотрим сколько постов и комментариев появлялось каждый час.

In [21]:
import datetime as dt

posts_per_hour = {}
comments_per_hour = {}

for row in ask_list:
    created_dt = dt.datetime.strptime(row[6], '%m/%d/%Y %H:%M')
    created_hour = created_dt.hour
    if created_hour in posts_per_hour:
        posts_per_hour[created_hour] += 1
    else:
        posts_per_hour[created_hour] = 1
        
    if created_hour in comments_per_hour:
        comments_per_hour[created_hour] += (row[4])
    else:
        comments_per_hour[created_hour] = (row[4])     

print(posts_per_hour)    

{9: 45, 13: 85, 10: 59, 14: 107, 16: 108, 23: 68, 12: 73, 17: 100, 15: 116, 21: 109, 20: 80, 2: 58, 18: 109, 3: 54, 5: 46, 19: 110, 1: 60, 22: 71, 8: 48, 4: 47, 0: 55, 6: 44, 7: 34, 11: 58}


In [15]:
print(comments_per_hour)

{9: 251, 13: 1253, 10: 793, 14: 1416, 16: 1814, 23: 543, 12: 687, 17: 1146, 15: 4477, 21: 1745, 20: 1722, 2: 1381, 18: 1439, 3: 421, 5: 464, 19: 1188, 1: 683, 22: 479, 8: 492, 4: 337, 0: 447, 6: 397, 7: 267, 11: 641}


И сколько в среднем приходилось комментариев на один пост в час.

In [20]:
avg_by_hour = []
temp_lis = []

for key in posts_per_hour:
    temp_list = [key, int(comments_per_hour[key]) / posts_per_hour[key]]
    avg_by_hour.append(temp_list)
    
print(avg_by_hour)    

[[9, 5.5777777777777775], [13, 14.741176470588234], [10, 13.440677966101696], [14, 13.233644859813085], [16, 16.796296296296298], [23, 7.985294117647059], [12, 9.41095890410959], [17, 11.46], [15, 38.5948275862069], [21, 16.009174311926607], [20, 21.525], [2, 23.810344827586206], [18, 13.20183486238532], [3, 7.796296296296297], [5, 10.08695652173913], [19, 10.8], [1, 11.383333333333333], [22, 6.746478873239437], [8, 10.25], [4, 7.170212765957447], [0, 8.127272727272727], [6, 9.022727272727273], [7, 7.852941176470588], [11, 11.051724137931034]]


Поменяем местами часы и количество средних постов.

In [23]:
swap_avg_by_hour = []
for row in avg_by_hour:
    first = row[0]
    second = row[1]
    row[1] = first
    row[0] = second
    swap_avg_by_hour.append(row)
    
print(swap_avg_by_hour)    

[[5.5777777777777775, 9], [14.741176470588234, 13], [13.440677966101696, 10], [13.233644859813085, 14], [16.796296296296298, 16], [7.985294117647059, 23], [9.41095890410959, 12], [11.46, 17], [38.5948275862069, 15], [16.009174311926607, 21], [21.525, 20], [23.810344827586206, 2], [13.20183486238532, 18], [7.796296296296297, 3], [10.08695652173913, 5], [10.8, 19], [11.383333333333333, 1], [6.746478873239437, 22], [10.25, 8], [7.170212765957447, 4], [8.127272727272727, 0], [9.022727272727273, 6], [7.852941176470588, 7], [11.051724137931034, 11]]


Отсортируем полученный список `swap_avg_by_hour` и определим пять наиболее удачных часов для вопросов.

In [36]:
sorted_swap = sorted(swap_avg_by_hour, reverse=True)
print('Лучшие часы для вопросов на HN')
for row in sorted_swap[:5]:
    hour_dt = dt.datetime.strptime(str(row[1]), '%H')
    print(dt.datetime.strftime(hour_dt, '%H:%M'), ': {:.2f} среднее количество комментов в час'.format(row[0]))

Лучшие часы для вопросов на HN
15:00 : 38.59 среднее количество комментов в час
02:00 : 23.81 среднее количество комментов в час
20:00 : 21.52 среднее количество комментов в час
16:00 : 16.80 среднее количество комментов в час
21:00 : 16.01 среднее количество комментов в час


Наибольшей популярностью пользуются посты, написанные с **15:00 до 16:00** - **38,59** комментариев в среднем. Это количество более чем в полтора раза превышает результаты второго места(посты с **02:00 до 03:00** - **23,81**.
Что интересно, посты, написаны уже с **16:00 до 17:00** гораздо менее популярны. Всего **16,8** в среднем.

Возможно такой всплеск по комеентариям между 15 и 16 часами вызван простой прокрастинацией. К этому времени работоспособность человека снижается(в предположении, что он работает с 9:00), и вместо работы человек начинает переключаться на что-то другое, в частности на HN.

В любом случае, 15:00 - время, на которое нужно ориентироваться, если хотите задать вопрос на HN.